## Data preparation WORK

**Change according to your configuration, and environment**

**Make sure there the 'scripts' folder and 'data' folder are presents.**

In [ ]:
!git clone https://github.com/gjuuuy/MT_base.git

In [ ]:
!cd MT_base

In [ ]:
%pip install datasets OpenNMT-py sentencepiece
%pip install --no-cache-dir https://github.com/RobotsMali-AI/rmai/releases/download/0.0.4/rmaipkg-0.0.4.tar.gz

In [ ]:
!python scripts/filter.py data/train.bam data/train.fr

In [ ]:
!python scripts/unigram.py data/train.bam.fil.txt data/train.fr.fil.txt

In [ ]:
!mv *.vocab data && mv *.model data

In [ ]:
!python scripts/subword.py data/source.model data/target.model data/train.bam.fil.txt data/train.fr.fil.txt
!python scripts/subword.py data/source.model data/target.model data/dev.bam data/dev.fr
!python scripts/subword.py data/source.model data/target.model data/test.bam data/test.fr

In [ ]:
!mv data/train.sub-src.txt data/train.sub.bam && mv data/train.sub-trg.txt data/train.sub.fr
!mv data/dev.sub-src.txt data/dev.sub.bam && mv data/dev.sub-trg.txt data/dev.sub.fr
!mv data/test.sub-src.txt data/test.sub.bam && mv data/test.sub-trg.txt data/test.sub.fr

## Model / Training Configuration

In [ ]:
import os

model_name = "bam2fr"
vocab_size = 50000

training_steps = 25000
valid_steps = int(training_steps / 5)
save_ckpt_freq = int(training_steps / 5)
warmup_steps = int(training_steps / 10)
reporting =  10 # int(training_steps/10)
GPU = 1 # TOGGLE for GPU

if(not os.path.exists(model_name)):
  os.makedirs(model_name)

config = f"""

# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: data/train.sub.bam
        path_tgt: data/train.sub.fr
        transforms: [filtertoolong]
    valid:
        path_src: data/dev.bam
        path_tgt: data/dev.fr
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: models/{model_name}/run/source.vocab
tgt_vocab: models/{model_name}/run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: data/source.model
tgt_subword_model: data/target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/{model_name}

# Stop training if it does not imporve after n validations
early_stopping: 3

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: {save_ckpt_freq}

# To save space, limit checkpoints to last n
keep_checkpoint: 2

seed: 3435

# Default: 100000 - Train the model to max n steps 
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: {training_steps}

# Default: 10000 - Run validation after n steps
valid_steps: {valid_steps}

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 1000
report_every: {reporting}

# Number of GPUs, and IDs of GPUs
#world_size: 2
#gpu_ranks: [0, 1]

# Batching
# bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 1024   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]

"""

if(GPU):
  config += """
world_size: 1
gpu_ranks: [0]
  """

with open(f"{model_name}/config.yaml", "w") as fp:
  fp.write(config)

In [ ]:
!onmt_build_vocab -c bam2fr/config.yaml -n_sample -1 --dump_samples # -1 full corpus, bpe, sentencepiece

In [ ]:
!onmt_build_vocab -config {model_name}/config.yaml -n_sample -1 -num_threads 2

## Model Evaluation

In [ ]:
!onmt_translate -model bam2fr/models/bam2fr_step_10000.pt -src data/test.sub-src.txt -output bam2fr/models/pred_10000.txt -gpu -1 -verbose

In [ ]:
!python scripts/desubword.py data/target.model bam2fr/models/pred_10000.txt

In [ ]:
# Sacrebleu testing CODE
bleu = !sacrebleu data/test.fr -i bam2fr/models/pred_10000.txt.desub.txt -m bleu -b -w 4
ter = !sacrebleu data/test.fr -i bam2fr/models/pred_10000.txt.desub.txt -m ter -b -w 4

print(bleu)
print(ter)